In [2]:
import xarray as xr
import pandas as pd

### DATA FROM:
https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r01/access/csv/

In [9]:
df = pd.read_csv('./ibtracs_data/ibtracs.NI.list.v04r01.csv', low_memory=False)

In [10]:
df = df.iloc[1:]
df = df[(df['SUBBASIN'] == 'AS') & (df['NATURE'] == 'TS')]

In [11]:
df.shape

(13322, 174)

In [14]:
# Convert the column to numeric, setting non-numeric values to NaN
df['WMO_WIND'] = pd.to_numeric(df['WMO_WIND'], errors='coerce')

# Drop rows where the 'WMO_WIND' has NaN values (i.e., non-integer values)
df = df.dropna(subset=['WMO_WIND'])
df.loc[:,'WMO_WIND'] = df['WMO_WIND'].astype(int)

In [30]:
df = df[['SID', 'ISO_TIME', 'LAT', 'LON', 'WMO_WIND', 'NEWDELHI_GRADE']]
df.loc[:, 'ISO_TIME'] = pd.to_datetime(df['ISO_TIME'])

df.set_index('ISO_TIME', inplace = True)

/Users/sg13n23/miniconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [19]:
for tc_id, tc_data in df.groupby('SID'):
    pass

In [29]:
df

,SID,ISO_TIME,LAT,LON,WMO_WIND,NEWDELHI_GRADE
45351,1982196N20069,1982-07-15 03:00:00,20.5,69.0,25.0,D
45354,1982196N20069,1982-07-15 12:00:00,21.0,68.0,25.0,D
45359,1982196N20069,1982-07-16 03:00:00,22.5,67.0,25.0,D
45475,1982274N13071,1982-10-01 03:00:00,13.5,71.0,25.0,D
45478,1982274N13071,1982-10-01 12:00:00,14.0,70.0,30.0,DD
...,...,...,...,...,...,...
61484,2022347N12074,2022-12-16 12:00:00,14.1,63.6,25.0,D
61486,2022347N12074,2022-12-16 18:00:00,14.1,62.9,25.0,D
61488,2022347N12074,2022-12-17 00:00:00,14.0,62.3,25.0,D
61489,2022347N12074,2022-12-17 03:00:00,14.0,62.0,25.0,D


In [38]:
count = 0
tcs_lmi_centred = []
for tc_id, tc_data in df.groupby('SID'):
    # The below if-statement is to check if any D or DD develops into a CS during it's lifetime.
    if any('CS' in item for item in set(df[df['SID'] == tc_id]['NEWDELHI_GRADE'].values)) == True:
        lmi_index = tc_data['SID'].idxmax()
        tc_data['TimeRelativeToLMI'] = (tc_data.index - lmi_index).total_seconds() / (3600 * 24)
        tcs_lmi_centred += [tc_data]
    else:
        count += 1

print('{} systems out of {} didn\'t mature into a CS'.format(count, len(list(df['SID'].unique()))))
tcs_lmi_centred = pd.concat(tcs_lmi_centred)

72 systems out of 113 didn't mature into a CS


In [40]:
tcs_lmi_centred.to_csv('CYCLONE_BEST_TRACK_AS_FROM_IBTRACS.csv')